In [9]:
import pandas as pd
import numpy as np

# Add src to path
import sys
sys.path.append('../src')

from preprocess.preprocess import partition_data
from preprocess.data_loader import prepare_chemical_data

2024-11-22 16:38:08,180 - INFO - TensorFlow version 2.13.0 available.
2024-11-22 16:38:08,181 - INFO - PyTorch version 2.0.1+cpu available.
2024-11-22 16:38:08,462 - INFO - Better speed can be achieved with apex installed from https://www.github.com/nvidia/apex .
2024-11-22 16:38:11,332 - INFO - Enabling RDKit 2024.03.6 jupyter extensions


In [2]:
combined_df = pd.read_csv("../data/processed/final_dataset.csv")
print(combined_df.head())

   Unnamed: 0  pert_dose  pert_time cell_mfc_name    pert_mfc_id  \
0           3  10.000000       24.0           PC3  BRD-K76674262   
1         134   3.333330       24.0           PC3  BRD-K76674262   
2         376   0.041200       24.0          A549  BRD-K76674262   
3         459   0.123457       24.0           PC3  BRD-K76674262   
4         570  10.000000       24.0         HEPG2  BRD-K76674262   

                                          sig_id  viability        pert_id  \
0                             POL001_PC3_24H:G19   0.372083  BRD-K76674262   
1                             POL001_PC3_24H:G20   0.410685  BRD-K76674262   
2  HOG002_A549_24H:BRD-K76674262-001-02-5:0.0412   0.825084  BRD-K76674262   
3                             POL001_PC3_24H:G23   0.698860  BRD-K76674262   
4                           POL001_HEPG2_24H:G19   0.516322  BRD-K76674262   

       cmap_name                                   canonical_smiles  ...  \
0  BRD-K76674262  COC(=O)C[C@](O)(CCCC(C)(C)O)

In [3]:
chem_df, viability_df, transcriptomics_df = partition_data(combined_df)

print(chem_df.head())
print(viability_df.head())
print(transcriptomics_df.head())

                                    canonical_smiles
0  COC(=O)C[C@](O)(CCCC(C)(C)O)C(=O)O[C@H]1[C@H]2...
1  COC(=O)C[C@](O)(CCCC(C)(C)O)C(=O)O[C@H]1[C@H]2...
2  COC(=O)C[C@](O)(CCCC(C)(C)O)C(=O)O[C@H]1[C@H]2...
3  COC(=O)C[C@](O)(CCCC(C)(C)O)C(=O)O[C@H]1[C@H]2...
4  COC(=O)C[C@](O)(CCCC(C)(C)O)C(=O)O[C@H]1[C@H]2...
   viability
0   0.372083
1   0.410685
2   0.825084
3   0.698860
4   0.516322
          1         2         3         4         5         6         7  \
0  1.691468  0.372236 -1.314469 -1.930791 -0.211148 -2.320093 -2.142510   
1  1.559223  0.320660 -1.116020 -1.846074 -0.547827 -1.222509 -1.477382   
2  2.041255  0.981665  0.003209 -0.021115  0.119147 -0.250142  0.275295   
3  0.966338 -0.417509 -0.633117  0.670356  0.388585 -0.558583 -1.565128   
4  1.471947  0.221686 -1.640204 -2.127661  0.868745 -2.315318 -0.876314   

          8         9        10  ...       673       674       675       676  \
0  0.876579  0.625576 -0.574647  ...  0.359634 -1.742530 -0.062960 -0.282

In [14]:
# Replace 'column_name' with the actual column name you want to convert to a list
smiles_list = chem_df['canonical_smiles'].tolist()
targets = viability_df["viability"].tolist()
print(len(smiles_list))
print(len(targets))

30097
30097


In [16]:
from preprocess.molecule_graph import mol_to_graph, collect_continuous_atom_features
from sklearn.preprocessing import StandardScaler

smiles = (
    "COC(=O)C[C@](O)(CCCC(C)(C)O)C(=O)O[C@H]1[C@H]2c3cc4OCOc4cc3CCN3CCC[C@]23C=C1OC"
)
smiles_list = [
    "COC(=O)C[C@](O)(CCCC(C)(C)O)C(=O)O[C@H]1[C@H]2c3cc4OCOc4cc3CCN3CCC[C@]23C=C1OC",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC1=CC(=C(C=C1)O)CC(=O)O",
    "CC(=O)O",  
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",
    "CC(C)(C)OC(=O)C1=CC(=C(C=C1)O)CC(=O)O",

]

continuous_atom_features = collect_continuous_atom_features(smiles_list)
scaler = StandardScaler()
scaler.fit(continuous_atom_features)

data = mol_to_graph(smiles, scaler)
print(data)

Start Exctarting features from SMILES
Data(x=[78, 15], edge_index=[2, 164], edge_attr=[164, 13])


In [17]:
chem_data_loader = prepare_chemical_data(smiles_list, targets, batch_size=16)

Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES
Start Exctarting features from SMILES


c:\Users\20191678\AppData\Local\anaconda3\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [19]:
from preprocess.data_loader import prepare_transcriptomics_data


transcriptomics_data_loader = prepare_transcriptomics_data(
        transcriptomics_df, targets, batch_size=32
    )

print(len(transcriptomics_data_loader))


941


c:\Users\20191678\AppData\Local\anaconda3\Lib\site-packages\torch_geometric\deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)
